In [1]:
import nltk
import pandas as pd

In [2]:
nltk.download("movie_reviews")

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/buckaroo/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [3]:
rows = []
for fileid in nltk.corpus.movie_reviews.fileids():
    rows.append({"text": nltk.corpus.movie_reviews.raw(fileid)})
df = pd.DataFrame(rows)
df.head()

,text
0,"plot : two teen couples go to a church party ,..."
1,the happy bastard's quick movie review \ndamn ...
2,it is movies like these that make a jaded movi...
3,""" quest for camelot "" is warner bros . ' firs..."
4,synopsis : a mentally unstable man undergoing ...


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)
vectorizer = vectorizer.fit(df['text'])
tfidf = vectorizer.transform(df['text'])
vocab = vectorizer.get_feature_names()
print(len(vocab))

21886


In [5]:
from corextopic import corextopic as ct

/home/buckaroo/nlp/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [6]:
anchors = []
model = ct.Corex(n_hidden=8, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

In [7]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: see, me, had, really, don, know, think, my, because, how
Topic #2: life, he is, both, never, it is, of his, that he, world, performance, to his
Topic #3: the first, the most, films, from the, many, by the, since, such, at the, while
Topic #4: comedy, funny, humor, jokes, laughs, funniest, the funniest, hilarious, the jokes, joke
Topic #5: young, opening, music, follow, portrayal, cinematography, mars, aspect, art, shown
Topic #6: murder, crime, thriller, police, killer, dead, the police, he has, turns, prison
Topic #7: plot, action, case, critique, the plot, suspense, none, blair witch, seem, cool
Topic #8: horror, horror film, scream, slasher, did last, horror films, scary, you did, williamson


In [8]:
# Anchors designed to nudge the model towards measuring specific genres
anchors = [
    ["action", "adventure"],
    ["drama"],
    ["comedy", "funny"],
    ["horror", "suspense"],
    ["animated", "animation"],
    ["sci fi", "alien"],
    ["romance", "romantic"],
    ["fantasy"]
]
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=8, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=3 # Tell the model how much it should rely on the anchors
)

In [9]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: action, adventure, the action, scenes, action sequences, where, action scenes, an action, action film, sequences
Topic #2: drama, performance, mother, director, both, while, and his, to his, role, performances
Topic #3: comedy, funny, jokes, laughs, humor, funny and, hilarious, very funny, gags, laugh
Topic #4: horror, really, think, had, me, did, how, see, because, were
Topic #5: animated, animation, disney, children, the animation, computer, adults, years, voice of, voice
Topic #6: alien, sci fi, effects, special effects, fi, aliens, sci, planet, special, earth
Topic #7: romantic, romance, she, love, with her, of her, that she, relationship, woman, romantic comedy
Topic #8: life, he is, fantasy, world, it is, that the, perhaps, point, does, through


In [10]:
print(len(anchors) )
def arr_to_topic(arr=[]):
    array = ""
    for s in arr:
        array+=s+" "
    
    return array   
        
print(arr_to_topic(anchors[0]))

8
action adventure 


In [13]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    # columns=["topic_{}".format(i+1) for i in range(8)]
    columns=[arr_to_topic(anchors[i]) for i in range(8)]
).astype(float)
topic_df.index = df.index
df = pd.concat([df, topic_df], axis=1)

In [14]:
df.sample(5, random_state=42)

,text,action adventure,drama,comedy funny,horror suspense,animated animation,sci fi alien,romance romantic,fantasy
1860,the verdict : spine-chilling drama from horror...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
353,""" the 44 caliber killer has struck again . "" ...",0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1333,in the company of men made a splash at the sun...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
905,"in the year 2029 , captain leo davidson ( mark...",0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1289,[note that followups are directed to rec . art...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
